In [1]:
!pip install ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

zsh:1: no matches found: ctransformers[cuda]


  Cloning https://github.com/huggingface/transformers to /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-req-build-8hkd85ef
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-req-build-8hkd85ef
  Resolved https://github.com/huggingface/transformers to commit d45f47ab7f7c31991bb98a0302ded59ab6adac31
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install bertopic

ERROR: Operation cancelled by user
^C


In [11]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [6]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=0,
    hf=True
)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [8]:
prompt = """<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.</s>
<|assistant|>"""


In [9]:
from bertopic.representation import TextGeneration
from bertopic import BERTopic

# Text generation with Zephyr
zephyr = TextGeneration(generator, prompt=prompt)
representation_model = {"Zephyr": zephyr}

# Topic Modeling
topic_model = BERTopic(representation_model=representation_model, verbose=True)

In [13]:
from datasets import load_dataset

dataset = load_dataset('hugginglearners/netflix-shows')['train']
descriptions = dataset['description']

In [14]:
topic_model.fit(descriptions[:100])

2024-03-07 05:46:39,636 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-03-07 05:46:52,045 - BERTopic - Embedding - Completed ✓
2024-03-07 05:46:52,047 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-07 05:47:13,919 - BERTopic - Dimensionality - Completed ✓
2024-03-07 05:47:13,930 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-07 05:47:13,954 - BERTopic - Cluster - Completed ✓
2024-03-07 05:47:14,008 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 3/3 [10:30<00:00, 210.29s/it]
2024-03-07 05:57:45,100 - BERTopic - Representation - Completed ✓


In [20]:
topic_info = topic_model.get_topic_info()

In [29]:
topic_distr, _ = topic_model.approximate_distribution(descriptions)

100%|██████████| 9/9 [00:04<00:00,  1.80it/s]


In [33]:
topic_model.visualize_distribution(topic_distr[8000])